# Koronaviruksen jätevesiseuranta

Jätevesitilasto © THL (lisenssillä [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.fi))

https://www.thl.fi/episeuranta/jatevesi/jatevesiseuranta_viikkoraportti.html

In [ ]:
from datetime import datetime
from jatevedet_fi_thl import *

In [ ]:
%config InlineBackend.figure_formats = ['png2x']  # png, png2x, jpg, svg
view_dpi = 90

In [ ]:
data = pd.read_csv("data/Koronaviruksen jätevesiseurannan viikkoraportti 2022-10-07.csv",
                   parse_dates=["Näytteen päivämäärä"], date_parser=datetime.fromisoformat)

In [ ]:
data.iloc[:20]

In [ ]:
df, dates, mittaukset_n, epvt_n, kaikki_puhdistamot = valmistele_data(data)

# Valitaan viikottain päivittyvät puhdistamot
puhdistamot = [
    "Helsinki, Viikinmäki",
    "Espoo, Suomenoja",
    "Jyväskylä, Nenäinniemi",
    "Tampere, Viinikanlahti",
    "Joensuu, Kuhasalo",
    "Oulu, Taskila",
    "Kuopio, Lehtoniemi",
    "Turku, Kakolanmäki",
    "Vaasa, Pått",
]
muut_puhdistamot = kaikki_puhdistamot.difference(puhdistamot)

# Järjestetään puhdistamot asiakasmäärien mukaan laskevasti
asiakasmaarat = df["Puhdistamon asiakasmäärä"][puhdistamot].max(axis=0)
puhdistamot = np.array(puhdistamot)[np.argsort(asiakasmaarat)[::-1]]

keskiarvot, painotetut_ka = trendien_keskiarvot(df, puhdistamot)

## Muut puhdistamot

Harvemmin kuin viikon välein päivittyvät puhdistamot ja ne puhdistamot joilla koronaviruksen jätevesiseuranta on päättynyt:

In [ ]:
print(*muut_puhdistamot, sep="\n")

## Epävarmuustekijät viimeaikaisissa näytteissä

Sarakkeen [kuvauksesta](https://www.thl.fi/episeuranta/jatevesi/jatevesiseuranta_viikkoraportti.html):

> Taulukkoriville on lisätty kirjainsymboli (a, b tai c), mikäli saatuun SARS-CoV-2 koronaviruksen RNA-tuloksen analysointiin liittyy yksi tai useampi tuloksen luotettavuutta heikentävä epävarmuustekijä.
> 
> * a = näytteen lämpötila yli 15 astetta näytteen saapuessa laboratorioon.
> * b = näytteessä on todettu määritystä häiritseviä tekijöitä (inhibitio).
> * c = yli 14vrk viive näytteen keräämisen ja analyysin aloituksen välillä, kertanäyte tai poikkeava keräysaika.
> * () = epävarmuustekijää ei ole testattu tai tieto puuttuu.


In [ ]:
df["Epävarmuustekijät"][puhdistamot].loc[dates].iloc[-1:-9:-1].T

In [ ]:
xlim = {
    "left": dates[-1] - np.timedelta64(12 * 28 + 2, "D"),
    "right": dates[-1] + np.timedelta64(2, "D"),
}

## Historiaan suhteutettujen mittaustulosten jakauma koko maassa

Kunkin puhdistamon virtaamakorjattuun dataan on sovitettu trendi (liukuva keskiarvo), joka tasoittaa viikottaista vaihtelua. Yllä tuloste ikkunan leveydestä kertoo, kuinka pitkältä aikaväliltä mittaustuloksia huomioidaan. Puhdistamon data normalisoidaan jakamalla jokainen virtaamakorjattu mittaustulos trendikäyrän maksimiarvolla. Puhdistamon mittaustuloksia verrataan siis sen omaan menneisyyteen – sen tähän asti pahimpaan epidemiatilanteeseen.

Normalisoinnin yksi tarkoitus on häivyttää alueiden väliset tuntemattomat erot vakio-olosuhteissa, jotka voivat ilmetä paikallisina eroina koronan leviämiskyvyssä tai sairastuneen tuottaman jätevesisignaalin voimakkuudessa. Lisäksi normalisoitujen mittaustulosten jakauman muutokset havainnollistavat hyvin paikallisten epidemiatilanteiden muutoksia ja "samantahtisuutta" koko maassa.

Alla olevassa kaaviossa on normalisoitujen mittaustulosten jakauma ajan funktiona.

In [ ]:
fig, ax = jakaumakaavio(df, dates, mittaukset_n, epvt_n, puhdistamot, 
                        keskiarvot=keskiarvot,
                        painotetut_ka=painotetut_ka,
                        xlim_kw=xlim,
                        dpi=view_dpi)
muita = len(muut_puhdistamot)
ax[0].plot(df.index, df["Normalisoitu RNA-lkm"][muut_puhdistamot], "k+", alpha=0.4,
           label=["(muiden puhdistamojen\nmittaustulokset)"] + [None] * (muita - 1))
ax[0].legend(title="Viikoittain seuratut puhdistamot", loc="upper left")

## Trendit

In [ ]:
trendikaavio(df, puhdistamot, xlim_kw=xlim)

In [ ]:
N = np.cumsum([0, 3, 3, 3])
assert N[-1] == len(puhdistamot)

for i in range(len(N) - 1):
    fig, ax = trendikaavio(df, puhdistamot[N[i]:N[i+1]],
                           xlim_kw=xlim, 
                           ylim_kw={"top": 1.2},
                           dpi=view_dpi)

## Puhdistamojen virtaaman vaihtelu

In [ ]:
virtaamat = df["Virtaama"][puhdistamot]
virtaamat_std = (virtaamat.interpolate(axis=0) - virtaamat.mean(axis=0)) / virtaamat.std(axis=0)
virtaamat_std_m = virtaamat_std.median(axis=1)

fig, ax = plt.subplots(figsize=(8, 5), dpi=view_dpi)
plt.plot(df.index, virtaamat_std, "C0-", alpha=.25)
plt.plot(df.index, virtaamat_std_m, "k-", label="mediaani")
plt.title("Standardoitu virtaama")
ax.xaxis.set_minor_locator(WeekdayLocator(byweekday=MO))
ax.grid(which="major", axis="both", c="k", alpha=0.1)
ax.grid(which="minor", axis="both", c="k", alpha=0.05)
plt.xticks(rotation=30, ha="right")
plt.xlim(**xlim)
plt.legend()